# Colab-BasicSR (pytorch lightning)

[This tutorial](https://towardsdatascience.com/from-pytorch-to-pytorch-lightning-a-gentle-introduction-b371b7caaf09), [this issue](https://stackoverflow.com/questions/65387967/misconfigurationerror-no-tpu-devices-were-found-even-when-tpu-is-connected-in)  and [this Colab](https://colab.research.google.com/github/PytorchLightning/pytorch-lightning/blob/master/notebooks/03-basic-gan.ipynb#scrollTo=3vKszYf6y1Vv) were very helpful. This Colab does support single-GPU, multi-GPU and TPU training. (TPU training will depend on the architecture. RFR does not work with TPUs because of .cuda() for example.)

Can use various loss functions and has the context_encoder discriminator as default. Currently there are only various inpainting generators from [my BasicSR fork](https://github.com/styler00dollar/Colab-BasicSR).

What is not included inside this Colab, but is included in [my normal BasicSR Colab](https://colab.research.google.com/github/styler00dollar/Colab-BasicSR/blob/master/Colab-BasicSR.ipynb):
- [edge-informed-sisr](https://github.com/knazeri/edge-informed-sisr/blob/master/src/models.py)
- [USRNet](https://github.com/cszn/KAIR/blob/master/models/network_usrnet.py)
- [OFT Dataloader](https://github.com/styler00dollar/Colab-BasicSR/tree/master/codes/data)
- Some loss functions, but most are here
- DiffAug / Mixup

What currently is here but not inside the other Colab:
- Custom mask loading
- New discriminators (EfficientNet, ResNeSt, Transformer)
- [AdamP](https://github.com/clovaai/AdamP)

In [ ]:
!nvidia-smi

In [ ]:
!git clone -b lightning https://github.com/styler00dollar/Colab-BasicSR

In [ ]:
#@title GPU
# create empty folders
!mkdir /content/hr
!mkdir /content/lr
!mkdir /content/val_hr
!mkdir /content/val_lr
 
!mkdir /content/masks
!mkdir /content/validation
!mkdir /content/data
!mkdir /content/logs/
 
#!pip install pytorch-lightning -U
# Hotfix, to avoid pytorch-lightning bug
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning
!pip install tensorboardX
 
# optional
!pip install efficientnet_pytorch
!pip install adamp

In [ ]:
#@title TPU  (restart runtime afterwards)
# create empty folders
!mkdir /content/masks
!mkdir /content/validation
!mkdir /content/data
!mkdir /content/logs/

#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev
#!pip install pytorch-lightning
!pip install lightning-flash

import collections
from datetime import datetime, timedelta
import os
import requests
import threading

_VersionConfig = collections.namedtuple('_VersionConfig', 'wheels,server')
VERSION = "xrt==1.15.0"  #@param ["xrt==1.15.0", "torch_xla==nightly"]
CONFIG = {
    'xrt==1.15.0': _VersionConfig('1.15', '1.15.0'),
    'torch_xla==nightly': _VersionConfig('nightly', 'XRT-dev{}'.format(
        (datetime.today() - timedelta(1)).strftime('%Y%m%d'))),
}[VERSION]
DIST_BUCKET = 'gs://tpu-pytorch/wheels'
TORCH_WHEEL = 'torch-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)
TORCH_XLA_WHEEL = 'torch_xla-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)
TORCHVISION_WHEEL = 'torchvision-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)

# Update TPU XRT version
def update_server_xrt():
  print('Updating server-side XRT to {} ...'.format(CONFIG.server))
  url = 'http://{TPU_ADDRESS}:8475/requestversion/{XRT_VERSION}'.format(
      TPU_ADDRESS=os.environ['COLAB_TPU_ADDR'].split(':')[0],
      XRT_VERSION=CONFIG.server,
  )
  print('Done updating server-side XRT: {}'.format(requests.post(url)))

update = threading.Thread(target=update_server_xrt)
update.start()

# Install Colab TPU compat PyTorch/TPU wheels and dependencies
!pip uninstall -y torch torchvision
!gsutil cp "$DIST_BUCKET/$TORCH_WHEEL" .
!gsutil cp "$DIST_BUCKET/$TORCH_XLA_WHEEL" .
!gsutil cp "$DIST_BUCKET/$TORCHVISION_WHEEL" .
!pip install "$TORCH_WHEEL"
!pip install "$TORCH_XLA_WHEEL"
!pip install "$TORCHVISION_WHEEL"
!sudo apt-get install libomp5
update.join()

#!pip install pytorch-lightning
!pip install git+https://github.com/PyTorchLightning/pytorch-lightning

!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py > /dev/null
!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev > /dev/null
!pip install pytorch-lightning > /dev/null

!pip install tensorboardX

# optional
!pip install efficientnet_pytorch
!pip install adamp

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('Google Drive connected.')

In [ ]:
#@title copy data somehow
"""
!mkdir '/content/data'
!mkdir '/content/data/images'
!cp "/content/drive/MyDrive/classification_v3.7z" "/content/data/images/data.7z"
%cd /content/data/images
!7z x "data.7z"
!rm -rf /content/data/images/data.7z
"""

# Training

In [ ]:
#@title config.yaml
%%writefile /content/Colab-BasicSR/code/config.yaml
name: template
scale: 16
gpus: 1
tpu_cores: 8
use_tpu: False
use_amp: True
use_swa: False
progress_bar_refresh_rate: 20
default_root_dir: '/content/drive/MyDrive/Colab-BasicSR/'

# Dataset options:
datasets:
  train:
    # DS_inpaint: hr is from dataroot_HR, loads masks
    # DS_inpaint_tiled: hr is from dataroot_HR, but images are grids (16x16 256px currently), loads masks
    # DS_inpaint_tiled_batch: hr is from dataroot_HR, but images are grids (16x16 256px currently) and processed as a batch (batch_size_DL), loads masks
    # DS_lrhr: loads lr from dataroot_LR and hr from dataroot_HR
    # DS_lrhr_batch_oft: loads grayscale-hr (3x3 400px) from dataroot_LR and generates lr by downscaling otf randomly

    mode: DS_lrhr_batch_oft # DS_inpaint | DS_inpaint_tiled | DS_inpaint_tiled_batch | DS_lrhr | DS_lrhr_batch_oft
    grayscale: True # If true, reads as 1-Channel image. If false, reads as 3-channel image. Currently only implemented for DS_lrhr_batch_oft
    dataroot_HR: '/content/hr' # Original, with a single directory. Inpainting will use this directory as source image.
    dataroot_LR: '/content/lr' # Original, with a single directory

    n_workers: 1 # 0 to disable CPU multithreading, or an integrer representing CPU threads to use for dataloading
    batch_size: 1
    batch_size_DL: 9 # If a batched/tiled dataloader is used, use this batch parameter instead. Leave normal batch_size at 1.
    HR_size: 400 # The resolution the network will get. Random crop gets applied if that resolution does not match.
    image_channels: 1 # number of channels to load images in

    masks: '/content/random_masks/train+test_inverted/'
    max_epochs: 200
    save_step_frequency: 5000 # also validation frequency

    # batch
    # If a tiled dataloader is used, specify image charactaristics. Random crop will not be applied. Maybe in the future.
    image_size: 400 # Size of one tile
    amount_tiles: 3 # Amount of tiles inside the merged grid image

    # if edge data is required
    canny_min: 100
    canny_max: 150

  val:
    dataroot_HR: '/content/drive/MyDrive/Colab-BasicSR/val_input_hr/'
    dataroot_LR: '/content/drive/MyDrive/Colab-BasicSR/val_input_lr/' # Inpainting will use this directory as input

path:
    pretrain_model_G: # '/content/Checkpoint_0_0_D.pth'
    checkpoint_path: # '/content/drive/MyDrive/Colab-BasicSR/Checkpoint_31_715000.ckpt'
    checkpoint_save_path: '/content/drive/MyDrive/Colab-BasicSR/'
    validation_output_path: '/content/drive/MyDrive/Colab-BasicSR/val/'
    log_path: '/content/drive/MyDrive/Colab-BasicSR/'

# Generator options:
network_G:
    finetune: False # Important for further rfr training. Apply that after training for a while. https://github.com/jingyuanli001/RFR-Inpainting/issues/33

    # ESRGAN:
    netG: RRDB_net # RRDB_net (original ESRGAN arch) | MRRDB_net (modified/"new" arch)
    norm_type: null
    mode: CNA
    nf: 64 # of discrim filters in the first conv layer (default: 64, good: 32)
    nb: 23 # (default: 23, good: 8)
    in_nc: 1 # of input image channels: 3 for RGB and 1 for grayscale
    out_nc: 1 # of output image channels: 3 for RGB and 1 for grayscale
    gc: 32
    group: 1
    convtype: Conv2D # Conv2D | PartialConv2D
    net_act: leakyrelu # swish | leakyrelu
    gaussian: true # true | false
    plus: false # true | false
    finalact: None #tanh # Test. Activation function to make outputs fit in [-1, 1] range. Default = None. Coordinate with znorm.
    upsample_mode: 'upconv'
    nr: 3

    # ASRGAN:
    #which_model_G: asr_resnet # asr_resnet | asr_cnn
    #nf: 64

    # PPON:
    #netG: ppon # | ppon
    ##norm_type: null
    #mode: CNA
    #nf: 64
    #nb: 24
    #in_nc: 3
    #out_nc: 3
    ##gc: 32
    #group: 1
    ##convtype: Conv2D #Conv2D | PartialConv2D

    # SRGAN:
    #netG: sr_resnet # RRDB_net | sr_resnet
    #norm_type: null
    #mode: CNA
    #nf: 64
    #nb: 16
    #in_nc: 3
    #out_nc: 3

    # SR:
    #netG: RRDB_net # RRDB_net | sr_resnet
    #norm_type: null
    #mode: CNA
    #nf: 64
    #nb: 23
    #in_nc: 3
    #out_nc: 3
    #gc: 32
    #group: 1

    # PAN:
    # netG: pan_net
    # in_nc: 3
    # out_nc: 3
    # nf: 40
    # unf: 24
    # nb: 16
    # self_attention: true
    # double_scpa: false

    # edge-informed-sisr
    #which_model_G: sisr
    #use_spectral_norm: True

    # USRNet
    #netG: USRNet
    #in_nc=4
    #out_nc=3
    #nc=[64, 128, 256, 512]
    #nb=2
    #act_mode='R'
    #downsample_mode='strideconv'
    #upsample_mode='convtranspose'

    # ----Inpainting Generators----
    # DFNet (batch_size: 2+, needs 2^x image input and validation) (2019)
    #netG: DFNet
    #c_img: 3
    #c_mask: 1
    #c_alpha: 3
    #mode: nearest
    #norm: batch
    #act_en: relu
    #act_de: leaky_relu
    #en_ksize: [7, 5, 5, 3, 3, 3, 3, 3]
    #de_ksize: [3, 3, 3, 3, 3, 3, 3, 3]
    #blend_layers: [0, 1, 2, 3, 4, 5]
    #conv_type: normal # partial | normal | deform
    

    # EdgeConnect (2019)
    #netG: EdgeConnect
    #use_spectral_norm: True
    #residual_blocks_edge: 8
    #residual_blocks_inpaint: 8
    #conv_type_edge: 'normal' # normal | partial | deform (has no spectral_norm)
    #conv_type_inpaint: 'normal' # normal | partial | deform

    # CSA (2019)
    #netG: CSA
    #c_img: 3
    #norm: 'instance'
    #act_en: 'leaky_relu'
    #act_de: 'relu'

    # RN (2020)
    #netG: RN
    #input_channels: 3
    #residual_blocks: 8
    #threshold: 0.8

    # deepfillv1 (2018)
    #netG:  deepfillv1

    # deepfillv2 (2019)
    #netG: deepfillv2
    #in_channels:  4
    #out_channels:  3
    #latent_channels:  64
    #pad_type:  'zero'
    #activation:  'lrelu'
    #norm: 'in'
    #conv_type: partial # partial | normal

    # Adaptive (2020)
    #netG: Adaptive
    #in_channels: 3
    #residual_blocks: 1
    #init_weights: True

    # Global (2020)
    #netG: Global
    #input_dim: 5
    #ngf: 32
    #use_cuda: True
    #device_ids: [0]

    # Pluralistic (2019)
    #netG: Pluralistic
    #ngf_E: 32
    #z_nc_E: 128
    #img_f_E: 128
    #layers_E: 5
    #norm_E: 'none'
    #activation_E: 'LeakyReLU'
    #ngf_G: 32
    #z_nc_G: 128
    #img_f_G: 128
    #L_G: 0
    #output_scale_G: 1
    #norm_G: 'instance'
    #activation_G: 'LeakyReLU'

    # crfill (2020)
    #netG: crfill
    #cnum: 48

    # DeepDFNet (experimental)
    #netG: DeepDFNet
    #in_channels:  4
    #out_channels:  3
    #latent_channels:  64
    #pad_type:  'zero'
    #activation:  'lrelu'
    #norm: 'in'

    # partial (2018)
    #netG: partial

    # DMFN (2020)
    #netG: DMFN
    #in_nc: 4
    #out_nc: 3
    #nf: 64
    #n_res: 8
    #norm: 'in'
    #activation: 'relu'

    # pennet (2019)
    #netG: pennet

    # LBAM (2019)
    #netG: LBAM
    #inputChannels: 4
    #outputChannels: 3

    # RFR (use_swa: false, no TPU) (2020)
    #netG: RFR
    #conv_type: partial # partial | deform

    # FRRN (2019)
    #netG: FRRN

    # PRVS (2019)
    #netG: PRVS

    # CRA (HR_size: 512) (2020)
    #netG: CRA
    #activation: 'elu'
    #norm: 'none'

    # atrous (2020)
    #netG: atrous

    # MEDFE (batch_size: 1) (2020)
    #netG: MEDFE

    # AdaFill (2021)
    #netG: AdaFill

# Discriminator options:
network_D:
    # VGG
    #netD: VGG
    #size: 256
    #in_nc: 3 #3
    #base_nf: 64
    #norm_type: 'batch'
    #act_type: 'leakyrelu'
    #mode: 'CNA'
    #convtype: 'Conv2D'
    #arch: 'ESRGAN'

    # VGG fea
    #netD: VGG_fea
    #size: 256
    #in_nc: 3
    #base_nf: 64
    #norm_type: 'batch'
    #act_type: 'leakyrelu'
    #mode: 'CNA'
    #convtype: 'Conv2D'
    #arch: 'ESRGAN'
    #spectral_norm: False
    #self_attention: False
    #max_pool: False
    #poolsize: 4


    #netD: VGG_128_SN

    # VGGFeatureExtractor
    #netD: VGGFeatureExtractor
    #feature_layer: 34
    #use_bn: False
    #use_input_norm: True
    #device: 'cpu'
    #z_norm: False

    # PatchGAN
    #netD: NLayerDiscriminator
    #input_nc: 3
    #ndf: 64
    #n_layers: 3
    #norm_layer: nn.BatchNorm2d
    #use_sigmoid: False
    #getIntermFeat: False
    #patch: True
    #use_spectral_norm: False

    # Multiscale
    #netD: MultiscaleDiscriminator
    #input_nc: 3
    #ndf: 64
    #n_layers: 3
    #norm_layer: nn.BatchNorm2d
    #se_sigmoid: False
    #num_D: 3
    #getIntermFeat: False

    # ResNet
    #netD: Discriminator_ResNet_128
    #in_nc: 3
    #base_nf: 64
    #norm_type: 'batch'
    #act_type: 'leakyrelu'
    #mode: 'CNA'
    
    #netD: ResNet101FeatureExtractor
    #use_input_norm: True
    #device: 'cpu'
    #z_norm: False

    # MINC
    #netD: MINCNet

    # Pixel
    #netD: PixelDiscriminator
    #input_nc: 3
    #ndf: 64
    #norm_layer: nn.BatchNorm2d

    # EfficientNet (3-channel input)
    #netD: EfficientNet
    #EfficientNet_pretrain: 'efficientnet-b0'

    # ResNeSt
    #netD: ResNeSt
    #ResNeSt_pretrain: 'resnest50' # ["resnest50", "resnest101", "resnest200", "resnest269"]

    # Transformer
    #netD: TranformerDiscriminator
    #img_size: 256
    #patch_size: 1
    #in_chans: 3
    #num_classes: 1
    #embed_dim: 64
    #depth: 7
    #num_heads: 4
    #mlp_ratio: 4.
    #qkv_bias: False
    #qk_scale: None
    #drop_rate: 0.
    #attn_drop_rate: 0.
    #drop_path_rate: 0.
    #hybrid_backbone: None
    #norm_layer: nn.LayerNorm

    netD: context_encoder

train:
    scheduler: AdamP # AdamP, Adam, SGDP
    lr: 0.0001

    # AdamP
    betas0: 0.9
    betas1: 0.999
    weight_decay: 1e-2

    # SGDP
    weight_decay: 1e-5
    momentum: 0.9
    nesterov: True

    # Losses:
    L1Loss_weight: 2

    # HFENLoss
    HFEN_weight: 0
    loss_f: L1CosineSim()
    kernel: 'log'
    kernel_size: 15
    sigma: 2.5
    norm: False

    # Elastic
    Elatic_weight: 0
    a: 0.2
    reduction_elastic: 'mean'

    # Relative L1
    Relative_l1_weight: 0
    eps: .01
    reduction_realtive: 'mean'

    # L1CosineSim (3-channel input)
    L1CosineSim_weight: 0
    loss_lambda: 5
    reduction_L1CosineSim: 'mean'

    # ClipL1
    ClipL1_weight: 0
    clip_min: 0.0
    clip_max: 10.0

    # FFTLoss
    FFTLoss_weight: 0
    loss_f: L1Loss
    reduction_fft: 'mean'

    OFLoss_weight: 0

    # GPLoss
    GPLoss_weight: 0
    trace: False
    spl_denorm: False

    # CPLoss
    CPLoss_weight: 0
    rgb: True
    yuv: True
    yuvgrad: True
    trace: False
    spl_denorm: False
    yuv_denorm: False

    StyleLoss_weight: 0

    # TVLoss
    TVLoss_weight: 0
    tv_type: 'tv'
    p: 1

    # PerceptualLoss (only single gpu)
    PerceptualLoss_weight: 1
    model: 'net-lin'
    net: 'alex'
    colorspace: 'rgb'
    spatial: False
    use_gpu: True # False for TPU training
    gpu_ids: [0]
    #model_path: None # todo

    # Contextual_Loss (3-channel input)
    Contexual_weight: 0
    layers_weights: {'conv_1_1': 1.0, 'conv_3_2': 1.0}
    crop_quarter: False
    max_1d_size: 100
    distance_type: 'cosine'
    b: 1.0
    band_width: 0.5
    use_vgg: True
    net_contextual: 'vgg19'
    calc_type: 'regular'

    # Style (3-channel input)
    StyleLoss_weight: 0

    stage1_weight: 0 # only if the network outputs 2 images, will use l1

    # Differentiable Augmentation for Data-Efficient GAN Training
    diffaug: False # not implemented
    policy: 'color,translation,cutout'

    # Metrics
    metrics: ['PSNR', 'MSE'] # PSNR | SSIM | AE | MSE

In [ ]:
%cd /content/Colab-BasicSR/code/
!python train.py

# Testing 

In [ ]:
#@title testing the model
%cd /content/Colab-BasicSR/code/
# output path currently defined in config file
# test file for inpainting, mask with green
!python test.py --data_input_folder '/content/val_lr/' --fp16_mode False --netG_pth_path '/content/drive/MyDrive/Colab-BasicSR/lightning/Checkpoint_39_100000_G.pth'

# Misc

In [ ]:
# Using pillow simd instead of normal pillow. Restart runtime after that.
!pip uninstall Pillow -y
!pip install Pillow-SIMD

In [ ]:
#@title creating tiles (with skip)
import cv2
import numpy
import glob
import shutil
import tqdm
import os
import PIL
from PIL import Image
import numpy as np

resize_method = 'PIL' #@param ["OpenCV", "PIL"] {allow-input: false}
grayscale = False #@param {type:"boolean"}

rootdir = '/content/data' #@param {type:"string"}
destination_dir = "/content/out/" #@param {type:"string"}
broken_dir = '/content/opencv_fail/' #@param {type:"string"}
 
files = glob.glob(rootdir + '/**/*.png', recursive=True)
files_jpg = glob.glob(rootdir + '/**/*.jpg', recursive=True)
files_jpeg = glob.glob(rootdir + '/**/*.jpeg', recursive=True)
files_webp = glob.glob(rootdir + '/**/*.webp', recursive=True)
files.extend(files_jpg)
files.extend(files_jpeg)
files.extend(files_webp)
err_files=[]

amount_tiles = 2 #@param
image_size = 256 #@param

filepos = 0
img_cnt = 0
filename_cnt = 0

if grayscale == True:
  tmp_img = numpy.zeros((amount_tiles*image_size,amount_tiles*image_size))
elif grayscale == False:
  tmp_img = numpy.zeros((amount_tiles*image_size,amount_tiles*image_size, 3))

with tqdm.tqdm(files) as pbar:
  while True:
      if grayscale == True:
        image = cv2.imread(files[filepos], cv2.IMREAD_GRAYSCALE)
      elif grayscale == False:
        image = cv2.imread(files[filepos])

      filepos += 1

      if image is not None:
        
        i = img_cnt % amount_tiles
        j = img_cnt // amount_tiles

        #####################################
        # resize with opencv
        if resize_method == "OpenCV":
          image = cv2.resize(image, (image_size,image_size), interpolation=cv2.INTER_AREA)

        # resize with PIL
        elif resize_method == "PIL":
          if grayscale == True:
            image = Image.fromarray(image)
            image = image.resize((image_size,image_size))
            image = np.asarray(image)
          if grayscale == False:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = Image.fromarray(image)
            image = image.resize((image_size,image_size), resample=PIL.Image.LANCZOS)
            image = np.asarray(image)
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        #####################################

        tmp_img[i*image_size:(i+1)*image_size, j*image_size:(j+1)*image_size] = image
        img_cnt += 1
      else:
        print(files[filepos])
        print(f'{broken_dir}/{os.path.basename(files[filepos])}')
        shutil.move(files[filepos], f'{broken_dir}/{os.path.basename(files[filepos])}')

      if img_cnt == (amount_tiles*amount_tiles):
        #cv2.imwrite(destination_dir+str(filename_cnt)+".jpg", tmp_img, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
        cv2.imwrite(destination_dir+str(filename_cnt)+".png", tmp_img)
        filename_cnt += 1
        img_cnt = 0
      pbar.update(1)

In [ ]:
#@title convert to onnx
#@markdown Make sure the input dimensions are correct. Maybe a runtime restart is needed if it complains about ``TypeError: forward() missing 1 required positional argument``. Make sure you only run the required cells.
from torch.autograd import Variable
model = CustomTrainClass()
checkpoint_path = '/content/Checkpoint_0_0.ckpt' #@param
output_path = '/content/output.onnx' #@param
model = model.load_from_checkpoint(checkpoint_path) # start training from checkpoint, warning: apperantly global_step will be reset to zero and overwriting validation images, you could manually make an offset
dummy_input = Variable(torch.randn(1, 1, 64, 64))

model.to_onnx(output_path, input_sample=dummy_input)

In [ ]:
#@title pip list with space
!pip list | tail -n +3 | awk '{print $1}' | xargs pip show | grep -E 'Location:|Name:' | cut -d ' ' -f 2 | paste -d ' ' - - | awk '{print $2 "/" tolower($1)}' | xargs du -sh 2> /dev/null | sort -hr

In [ ]:
#@title tiling script
import cv2
import numpy
import glob
import shutil
import tqdm
import os
from multiprocessing.pool import ThreadPool as ThreadPool

rootdir = 'C:\\Users\\x\\Desktop\\Untitled Folder\\data\\' #@param {type:"string"}
destination_dir = "C:\\Users\\x\\Desktop\\Untitled Folder\\" #@param {type:"string"}
broken_dir = 'C:\\Users\\x\\Desktop\\Untitled Folder\\' #@param {type:"string"}
threads = 2 #@param
tile_size = 256 #@param

files = glob.glob(rootdir + '/**/*.png', recursive=True)
files_jpg = glob.glob(rootdir + '/**/*.jpg', recursive=True)
files_jpeg = glob.glob(rootdir + '/**/*.jpeg', recursive=True)
files_webp = glob.glob(rootdir + '/**/*.webp', recursive=True)
files.extend(files_jpg)
files.extend(files_jpeg)
files.extend(files_webp)
err_files=[]

pool = ThreadPool(threads)

def tiling(f):
  image = cv2.imread(f)
  if image is not None:
      counter = 0

      x = image.shape[0]
      y = image.shape[1]

      x_amount = x // tile_size
      y_amount = y // tile_size

      for i in range(x_amount):
        for j in range(y_amount):
          crop = image[i*tile_size:(i+1)*tile_size, (j*tile_size):(j+1)*tile_size]
          cv2.imwrite(os.path.join(destination_dir, os.path.splitext(os.path.basename(f))[0] + str(counter) + ".png"), crop)
          counter += 1

    else:
        print(f'Broken file: {os.path.basename(f)}')
        shutil.move(f, f'{broken_dir}/{os.path.basename(f)}')

        
pool.map(tiling, files)

# Getting pretrained models

In [ ]:
#@title download rfr paris model and fix state_dict
# rfr paris
%cd /content/
!gdown --id 1jnUb-EvBw9DcwyWUQyWDdN9o42BPH7uT

#https://discuss.pytorch.org/t/dataparallel-changes-parameter-names-issue-with-load-state-dict/60211
import torch
from collections import OrderedDict
state_dict = torch.load("/content/checkpoint_paris.pth", map_location='cpu')
new_state_dict = OrderedDict()

for k, v in state_dict['generator'].items():
  if k == 'Pconv1.weight':
      name = 'conv1.weight'
  elif k == 'Pconv2.weight':
      name = 'conv2.weight'
  elif k == 'Pconv21.weight':
      name = 'conv21.weight'
  elif k == 'Pconv22.weight':
      name = 'conv22.weight'
  else:
      name = k

  new_state_dict[name] = v

torch.save(new_state_dict, '/content/converted.pth')

In [ ]:
#@title downloading places2 dfnet
%cd /content/
!gdown --id 1SGJ_Z9kpchdnZ3Qwwf4HnN-Cq-AeK7vH # dfnet places2

# Misc

A summary of all interesting inpainting generators that are not trainable with my code.

--------------------------------------------------

``Broken generators:``

Generators that are not included here since I can't seem to make them work properly:

AOT-GAN (2021): [researchmm/AOT-GAN-for-Inpainting](https://github.com/researchmm/AOT-GAN-for-Inpainting)

    Couldn't get generator working.

PenNet [no AMP] (2019): [researchmm/PEN-Net-for-Inpainting](https://github.com/researchmm/PEN-Net-for-Inpainting/)

    Always outputs white for some reason.

CRA [no AMP] (2019): [wangyx240/High-Resolution-Image-Inpainting-GAN](https://github.com/wangyx240/High-Resolution-Image-Inpainting-GAN)

    Likes to create the color pink.

Global [no AMP] (2020): [SayedNadim/Global-and-Local-Attention-Based-Free-Form-Image-Inpainting](https://github.com/SayedNadim/Global-and-Local-Attention-Based-Free-Form-Image-Inpainting)

    Always outputs white for some reason.

crfill (2020): [zengxianyu/crfill](https://github.com/zengxianyu/crfill)

    No clear instructions/code result in broken results. Unreleased training code makes a correct implementation harder.

--------------------------------------------------

``Non-Pytorch generators:``

PSR-Net (2020): [sfwyly/PSR-Net](https://github.com/sfwyly/PSR-Net)

    Uses Tensorflow 2

co-mod-gan (2021): [zsyzzsoft/co-mod-gan](https://github.com/zsyzzsoft/co-mod-gan)

    Has a web demo and (a broken link to a) docker. Relies on Tensorflow 1.15 / StyleGAN2 code. A Colab by me for this can be found inside https://github.com/styler00dollar/Colab-co-mod-gan.

Diverse-Structure-Inpainting (2021): [USTC-JialunPeng/Diverse-Structure-Inpainting](https://github.com/USTC-JialunPeng/Diverse-Structure-Inpainting)

    Tensorflow 1

R-MNet (2021): [Jireh-Jam/R-MNet-Inpainting-keras](https://github.com/Jireh-Jam/R-MNet-Inpainting-keras)

    Not sure if there is much new and interesting stuff.

Hypergraphs (2021): [GouravWadhwa/Hypergraphs-Image-Inpainting](https://github.com/GouravWadhwa/Hypergraphs-Image-Inpainting)

    Uses custom conv layer (that is implemented with tensorflow). It sounds interesting, but I got errors when I tried to port it to pytorch.

PEPSI (2019): [Forty-lock/PEPSI-Fast_image_inpainting_with_parallel_decoding_network](https://github.com/Forty-lock/PEPSI-Fast_image_inpainting_with_parallel_decoding_network)

    The net dcpV2 uses.

Region (2019): [vickyFox/Region-wise-Inpainting](https://github.com/vickyFox/Region-wise-Inpainting)

--------------------------------------------------

``Pytorch generators that I never tested:``

Edge-LBAM (2021): [wds1998/Edge-LBAM](https://github.com/wds1998/Edge-LBAM)

VCNET (2020): [birdortyedi/vcnet-blind-image-inpainting](https://github.com/birdortyedi/vcnet-blind-image-inpainting)

    Blind image inpainting without masks.

DFMA (2020): [mprzewie/dmfa_inpainting](https://github.com/mprzewie/dmfa_inpainting)

GIN (2020): [rlct1/gin-sg](https://github.com/rlct1/gin-sg) and [rlct1/gin](https://github.com/rlct1/gin)

StructureFlow (2019): [RenYurui/StructureFlow](https://github.com/RenYurui/StructureFlow)

    Needs special files.

GMCNN (2018): [shepnerd/inpainting_gmcnn](https://github.com/shepnerd/inpainting_gmcnn)

    The net dcpV1 used iirc.

ShiftNet (2018): [Zhaoyi-Yan/Shift-Net_pytorch](https://github.com/Zhaoyi-Yan/Shift-Net_pytorch)

--------------------------------------------------
``Soon:``

ICT (2021): [raywzy/ICT](https://github.com/raywzy/ICT)

--------------------------------------------------

``No training code:``

SC-FEGAN (2019): [run-youngjoo/SC-FEGAN](https://github.com/run-youngjoo/SC-FEGAN)
